<a href="https://colab.research.google.com/github/Jean-Carlosms/-Criando-um-Sistema-de-Recomenda-o-por-Imagens-Digitais/blob/main/Criando_um_Sistema_de_Recomenda%C3%A7%C3%A3o_por_Imagens_Digitais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 39.0 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import faiss
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import requests
from io import BytesIO
import pickle

# Classe para extração de características
class FeatureExtractor:
    def __init__(self, device="cpu"):
        self.device = device
        self.model = models.resnet50(pretrained=True).to(self.device)
        self.model.fc = torch.nn.Identity()
        self.model.eval()
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def extract(self, img):
        image = self.transform(img).unsqueeze(0).to(self.device)
        with torch.no_grad():
            features = self.model(image)
        return features.cpu().squeeze().numpy()

# Baixar imagem de URL
def download_image(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    return image

# Criar e salvar o índice FAISS
def build_index(image_folder, index_path="faiss_index.bin", metadata_path="image_paths.pkl"):
    extractor = FeatureExtractor(device="cuda" if torch.cuda.is_available() else "cpu")

    image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    feature_list = []

    for img_path in image_paths:
        image = Image.open(img_path).convert("RGB")
        features = extractor.extract(image)
        feature_list.append(features)

    features_matrix = np.array(feature_list, dtype=np.float32)
    index = faiss.IndexFlatL2(features_matrix.shape[1])
    index.add(features_matrix)

    faiss.write_index(index, index_path)
    with open(metadata_path, "wb") as f:
        pickle.dump(image_paths, f)

    return index, image_paths

# Carregar o índice FAISS salvo
def load_index(index_path="faiss_index.bin", metadata_path="image_paths.pkl"):
    index = faiss.read_index(index_path)
    with open(metadata_path, "rb") as f:
        image_paths = pickle.load(f)
    return index, image_paths

# Buscar imagens similares
def search_similar(image_url, index, image_paths, extractor, k=5):
    query_image = download_image(image_url)
    query_feature = extractor.extract(query_image).reshape(1, -1)
    distances, indices = index.search(query_feature, k)
    return [image_paths[i] for i in indices[0]]

# Exemplo de uso
if __name__ == "__main__":
    image_folder = "dataset_images"
    index_path = "faiss_index.bin"
    metadata_path = "image_paths.pkl"

    # Construir ou carregar o índice
    if not os.path.exists(index_path) or not os.path.exists(metadata_path):
        index, image_paths = build_index(image_folder, index_path, metadata_path)
    else:
        index, image_paths = load_index(index_path, metadata_path)

    # Criar extrator de características
    extractor = FeatureExtractor(device="cuda" if torch.cuda.is_available() else "cpu")

    # Buscar imagens similares
    #query_image_url = "https://b.fssta.com/uploads/application/soccer/headshots/713.vresize.350.350.medium.34.png"
    results = search_similar(query_image_url, index, image_paths, extractor)

    print("Imagens similares encontradas:")
    for img in results:
        print(img)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 91.4MB/s]


FileNotFoundError: [Errno 2] No such file or directory: 'dataset_images'